In [159]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib
import re
import pymongo
import googletrans
from googletrans import Translator
import numpy as np
import geopy
import translators as ts
from geopy.geocoders import Nominatim 
geopy.geocoders.options.default_user_agent = "my_cool_app"
from country_list import countries_for_language

'hello.'

## Scraping

In [2]:
def get_html(url):
    result = requests.get(url)
    return BeautifulSoup(result.text, "html.parser"), result.text

def download_image(url, file_path, file_name):
    full_path = file_path + file_name + '.jpg'
    urllib.request.urlretrieve(url, full_path)

In [3]:
def parse_listing(url="https://www.boot24.com/boot/1976-ketch-baarda-zeegrundel-8222682/"):
    html, source = get_html(url)

    details = html.find("tbody", class_="datatable-section")
    description = html.find("div", class_="detail-description").text

    attributes = {}

    for row in details.find_all("tr"):
        attribute_name = row.find(class_="datatable-title").text
        attribute_value = row.find(class_="datatable-value").text

        attributes[attribute_name] = attribute_value

    
    return description, attributes

In [ ]:
page = 1
# id_ = 1
listings = []

while True:
    url = f"https://www.boot24.com/boote/typ-segelboote/seite-{page}/"
    page_html, page_source = get_html(url)

    if page_html.find(class_="disabled next"):
        print("Finished scraping")
        break

    print(f"Scraping page {page}")

    for listing in page_html.find_all("li", class_="listing-result listing-result-row"):
        name_ = listing.find(class_="name").text
        price = listing.find(class_="price").text
        location = listing.find(class_="location").text
        offered_by = listing.find(class_="offered-by").text
        link = listing.find(class_="main-link")["href"]
        id_ = listing.find(class_="main-link")["data-reporting-click-product-id"]
        # img_url = re.findall(r"https://images.boatsgroup\.com/resize.+?" + str(id_) + r".+?\.jpg", page_source)[0]
        try:
            img_url = re.findall(r"((https://images.boatsgroup\.com/resize/\d/\d+/\d+/" + str(id_) + r"_.+?)(jpg|png))", page_source)[0][0]
            download_image(img_url, "images\\", str(id_))
        except:
            img_url = ""

        listing_url = f"https://www.boot24.com{link}"

        # listing_html = get_html(listing_url)
        # try:
        #     img_url = listing.find("img")["src"]
        # if img_url != "":

        # except:
        #     pass

        description, attributes = parse_listing(listing_url)


        listings.append({
            "name":name_, 
            "price":price, 
            "location":location, 
            "offerer":offered_by, 
            "link":link, 
            "id":id_, 
            "img_url":img_url, 
            "listing_url":listing_url, 
            "description":description} 
            | attributes
        )

        # id_ +=1

    page += 1

## Data Preparation

In [193]:
df = pd.DataFrame(listings)
# df.to_csv("data_raw.csv")
df.drop(columns=["listing_url", "link", "img_url", "listing_url", "Rumpfgarantie", "HIN"], inplace=True)
df = df.rename(columns={"Jahr":"year", "Hersteller":"manufacturer", "Modell":"model", "Klasse":"category", "Länge":"length", "Treibstoff":"fuel_type", "Rumpfmaterial":"hull_material", "Rumpfform":"hull_shape"})
df.replace({"Diesel":"diesel", "Anderes":"other", "Elektrisch":"electrical", "Benzin":"petrol"}, inplace=True)
df.replace({"Stahl":"steel", "GFK / Fiberglass / Polyester": "GFK / fiberglas / polyester", "Ferrozement":"ferrocement", "Komposit":"composite", "Anderen Materialien":"other", 'PVC (Kunststoff)':"PVC", "Holz":"wood"}, inplace=True)

In [194]:
df = df.loc[df["price"] != 'Preis auf Anfrage']

In [195]:
df["price"] = df["price"].str.replace(".", "").str.replace("€", "").astype(float)
df["length"] = df["length"].str.replace("m", "").str.replace(",", ".").astype(float)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10948\1653832274.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["price"] = df["price"].str.replace(".", "").str.replace("€", "").astype(float)


In [196]:
df.location = df.location.str.lower()

In [197]:
countries_de = np.array([i[1].lower() for i in countries_for_language("de")])
countries = list(map(lambda x: x.lower(), ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe']))

## Translation

In [199]:
nm = Nominatim()

In [200]:
def get_country(x):

    res = countries_de[np.isin(countries_de, x.split())]

    if len(res) == 0:
        return np.nan
    else:
        return res[0]

In [201]:
df["country"] = df["location"].apply(get_country)
df.loc[df["country"] .isna(), "country"] = df.loc[df["country"] .isna(), "country"].apply(lambda x: nm.geocode(x, language="de")[0].split()[-1])

In [202]:
df.country = df.country.str.lower()

In [ ]:
translator = Translator()
df["description"] = df["description"].apply(lambda x: translator.translate(x, dest="en").text)

In [215]:
x = translator.translate(list(df["description"])) # 5m

In [218]:
df_ = df.copy()
df_["description"] = [i.text for i in x]

In [221]:
df_["country"] = [i.text for i in translator.translate(list(df["country"]))]

In [225]:
df_["country"] = df_.country.replace({"deutschland":"germany"}).str.lower()

In [230]:
df_.to_csv("data_clean.csv")